In [37]:
import numpy as np
import scipy.io as sio
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from cplAE_MET.utils.analysis_helpers import load_exp_output
from cplAE_MET.utils.utils import  get_all_1d_mask
from collections import Counter
from sklearn.model_selection import KFold

import pandas as pd

In [8]:
output = load_exp_output(exp_name="MET_10k_sratified_50met", pkl_file="Results_trial_144.pkl")
output.keys()

dict_keys(['XT', 'XM', 'XE', 'XrT', 'XrE', 'XrM', 'XrM_me_paired', 'XrE_me_paired', 'rec_arbor_density', 'zm', 'ze', 'zt', 'zme_paired', 'is_t_1d', 'is_e_1d', 'is_m_1d', 'cluster_id', 'gene_ids', 'e_features', 'specimen_id', 'cluster_label', 'merged_cluster_label_at40', 'merged_cluster_label_at50', 'merged_cluster_label_at60', 'cluster_color', 'platform', 'class', 'class_id', 'group', 'subgroup', 'hist_ax_de_api_bas', 'M_nmf_total_vars_ax', 'M_nmf_total_vars_de', 'M_nmf_total_vars_api', 'M_nmf_total_vars_bas', 'M_nmf_components_ax', 'M_nmf_components_de', 'M_nmf_components_api', 'M_nmf_components_bas', 'train_ind', 'val_ind'])

In [9]:
masks=get_all_1d_mask(output)
masks.keys()

dict_keys(['is_t_1d', 'is_e_1d', 'is_m_1d', 'is_me_1d', 'is_te_1d', 'is_tm_1d', 'is_met_1d', 'is_t_only_1d', 'is_e_only_1d', 'is_m_only_1d', 'is_te_only_1d', 'is_tm_only_1d', 'is_me_only_1d', 'is_patchseq_1d', 'is_fMOST_1d', 'is_ME_1d', 'is_EM_1d'])

In [44]:
df = pd.DataFrame(columns=["t_clusters", "mt_class_acc", "mt_class_acc_std"])

for i, level in enumerate(["cluster_label", "merged_cluster_label_at40", "merged_cluster_label_at50"]):
    X = output['XM'][masks['is_tm_1d']]
    y = np.array([i.rstrip() for i in output[level][masks['is_tm_1d']]])
    n_types = len(Counter(np.array([i.rstrip() for i in output[level][masks['is_t_1d']]])))

    kf = KFold(n_splits=10, random_state=None, shuffle=False)
    acc = []
    for i, (train_index, test_index) in enumerate(kf.split(X)):
        X_train = X[train_index]
        y_train = y[train_index]
        X_test = X[test_index]
        y_test = y[test_index]

        rf = RandomForestClassifier()
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)
        acc.append(accuracy_score(y_test, y_pred))

    df.loc[i, "t_clusters"] = n_types
    df.loc[i, "mt_class_acc"] = "{:.2f}".format(np.mean(acc)*100)
    df.loc[i, "mt_class_acc_std"] = "{:.2f}".format(np.std(acc)*100)
    print(df)

  t_clusters mt_class_acc mt_class_acc_std
9         93        36.86             2.37
  t_clusters mt_class_acc mt_class_acc_std
9         59        53.23             2.95
  t_clusters mt_class_acc mt_class_acc_std
9         50        58.58             2.15


In [43]:
df

,t_clusters,mt_class_acc,mt_class_acc_std
9,50,57.66,2.92


In [36]:
np.std(acc)

0.031358051810051454